In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 65kB/s 
     |████████████████████████████████| 204kB 45.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=82b7d810d133ec1c02c4665d0926ee6597c1070a55ad97c4b575cedb565d047e
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Organización de Datos 75.06. Primr Cuatrimestre de 2016. Examen parcial, primera oportunidad

### 1) UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2019. (***) (15 pts)

In [4]:
# (driver_id, car_id, trip_id, customer_id, date, distance_traveled)
trip_summary = [ (1000, 2000, 3000, 4001, '2019-01-01', 1.5),
                (1001, 2020, 3001, 4000, '2019-04-02', 1.2),
                (1002, 2019, 3002, 4000, '2019-04-03', 3.3),
                (1003, 2018, 3003, 4001, '2019-04-04', 1.4),
                (1004, 2017, 3004, 4002, '2019-04-05', 3.7),
                (1005, 2016, 3005, 4000, '2019-01-06', 2.5),
                (1006, 2015, 3006, 4001, '2019-04-07', 1.1),
                (1007, 2014, 3007, 4002, '2019-04-08', 3.2),
                (1008, 2013, 3008, 4000, '2019-03-09', 1.5),
                (1009, 2012, 3009, 4000, '2019-01-08', 2.5),
                (1010, 2011, 3010, 4002, '2019-04-07', 3.3),
                (1001, 2010, 3011, 4002, '2019-01-06', 1.4),
                (1002, 2009, 3012, 4004, '2019-03-05', 2.5),
                (1003, 2008, 3013, 4004, '2019-03-01', 1.6),
                (1004, 2007, 3014, 4000, '2019-04-01', 3.7),
                (1005, 2006, 3015, 4005, '2019-04-05', 2.8),
                (1006, 2005, 3016, 4005, '2019-04-05', 1.9),
                (1007, 2004, 3017, 4005, '2019-04-05', 2.8),
                (1008, 2003, 3018, 4005, '2019-01-01', 3.7),
                (1001, 2002, 3019, 4005, '2019-02-02', 2.6),
                (1002, 2001, 3020, 4005, '2019-02-03', 1.1),
                (1003, 2021, 3021, 4003, '2019-04-04', 2.2),
                (1010, 2022, 3022, 4001, '2019-04-09', 3.3)
                
]

In [6]:
trip_summaryRDD = sc.parallelize(trip_summary)
trip_summaryRDD.collect()

[(1000, 2000, 3000, 4001, '2019-01-01', 1.5),
 (1001, 2020, 3001, 4000, '2019-04-02', 1.2),
 (1002, 2019, 3002, 4000, '2019-04-03', 3.3),
 (1003, 2018, 3003, 4001, '2019-04-04', 1.4),
 (1004, 2017, 3004, 4002, '2019-04-05', 3.7),
 (1005, 2016, 3005, 4000, '2019-01-06', 2.5),
 (1006, 2015, 3006, 4001, '2019-04-07', 1.1),
 (1007, 2014, 3007, 4002, '2019-04-08', 3.2),
 (1008, 2013, 3008, 4000, '2019-03-09', 1.5),
 (1009, 2012, 3009, 4000, '2019-01-08', 2.5),
 (1010, 2011, 3010, 4002, '2019-04-07', 3.3),
 (1001, 2010, 3011, 4002, '2019-01-06', 1.4),
 (1002, 2009, 3012, 4004, '2019-03-05', 2.5),
 (1003, 2008, 3013, 4004, '2019-03-01', 1.6),
 (1004, 2007, 3014, 4000, '2019-04-01', 3.7),
 (1005, 2006, 3015, 4005, '2019-04-05', 2.8),
 (1006, 2005, 3016, 4005, '2019-04-05', 1.9),
 (1007, 2004, 3017, 4005, '2019-04-05', 2.8),
 (1008, 2003, 3018, 4005, '2019-01-01', 3.7),
 (1001, 2002, 3019, 4005, '2019-02-02', 2.6),
 (1002, 2001, 3020, 4005, '2019-02-03', 1.1),
 (1003, 2021, 3021, 4003, '2019-04

In [21]:
def get_month (fecha):
  lista_aux = []
  lista_aux = fecha.split('-')
  return int(lista_aux[1])
#variable = get_month('2019-02-03')
#variable
trip_summaryRDD_April = (((trip_summaryRDD.filter(lambda x: get_month(x[4])==4)).map(lambda x: (x[0],x[5]))).reduceByKey(lambda x,y: x+y)).reduce(lambda x,y: x if x[1]>y[1] else y)

(1004, 7.4)